In [2]:
import pandas as pd

import src.paths as paths
from src.utils import clean_col_names, currency_formating

In [3]:
print("hello")

hello


In [3]:
journal = pd.read_excel(io=paths.JOURNAL_RAW_FILE_PATH, dtype=str)
plan_de_compte = pd.read_excel(io=paths.PLAN_DE_COMPTE_RAW_FILE_PATH, dtype=str)

In [4]:
journal.columns = clean_col_names(journal)
plan_de_compte.columns = clean_col_names(plan_de_compte)

In [5]:
for col in ["DEBIT", "CREDIT"]:
    journal[col] = journal[col].astype(float)

for col in ["DATE", "ECHEANCE"]:
    journal[col] = pd.to_datetime(journal[col], format="%d/%m/%Y")
    
journal["AAAAMM"] = journal["DATE"].dt.strftime("%Y%m")

In [6]:
# Filtre pour ne conserver que les charges
masque_charge = journal["COMPTE"].str[0] == "6"
charges = journal.loc[masque_charge].reset_index(drop=True)

In [7]:
charges_plan_de_compte = charges.merge(
    right=plan_de_compte,
    how="left",
    on="COMPTE",
    validate="m:1",
)

In [8]:
charges_plan_de_compte["DEFINITION"].isna().value_counts()

DEFINITION
False    702
Name: count, dtype: int64

In [10]:
charges_plan_de_compte.to_excel("exemple.xlsx", index=False)

In [11]:
years = sorted(charges_plan_de_compte["AAAAMM"].unique().tolist())
v = [currency_formating] * len(years)
dict_formating = dict(zip(years, v))
dict_formating["TOTAL"] = currency_formating

In [1]:
charges_plan_de_compte

NameError: name 'charges_plan_de_compte' is not defined

# Charges par catégorie x période

In [12]:
(
    charges_plan_de_compte.groupby(
        ["CATEGORIE", "AAAAMM"], as_index=False, dropna=False
    )
    .agg({"DEBIT": "sum"})
    .pivot_table(index="CATEGORIE", columns="AAAAMM", values="DEBIT")
    .reset_index()
    .assign(TOTAL=lambda df_: df_[years].sum(axis=1))
    .sort_values("TOTAL", ascending=False, ignore_index=True)
    .to_excel(paths.EXCEL_OUTPUT_FILE_PATH, index=False)
    # .style.format(dict_formating, na_rep="")
)

# Charges par catégorie x compte x période

In [14]:
(
    charges_plan_de_compte.groupby(
        ["CATEGORIE", "DEFINITION", "COMPTE", "AAAAMM"], as_index=False, dropna=False
    )
    .agg({"DEBIT": "sum"})
    .pivot_table(
        index=["CATEGORIE", "DEFINITION", "COMPTE"],
        columns="AAAAMM",
        values="DEBIT",
    )
    .reset_index()
    .assign(TOTAL=lambda df_: df_[years].sum(axis=1))
    .sort_values("TOTAL", ascending=False, ignore_index=True)
    .to_excel(paths.EXCEL_OUTPUT_FILE_PATH, index=False)
    # .style.format(dict_formating, na_rep="")
)


In [ ]:
charges_plan_de_compte.query("COMPTE == '601000'").sort_values("DATE", ignore_index=True)